In [1]:
import xmltodict
import json
from enum import Enum

In [2]:
class Size(Enum):
    L = "Large"
    M = "Medium"
    S = "Small"
    G = "Gargantuan"
    T = "Tiny"

In [3]:
class Stat(Enum):
    Con = "consitution"
    Int = "intelligence"
    Str = "strength"
    Dex = "dexterity"
    Wis = "wisdom"
    Cha = "charisma"

In [4]:
with open('../data/191328_Complete.xml', 'r') as file:
    xml = file.read()
json_data = xmltodict.parse(xml)

In [5]:
json_copy = json_data.copy()

In [6]:
monsters = json_copy['compendium']['monster']

In [7]:
def getMonsterWithName(name, copy=True):
    monster = [monster for monster in monsters if monster['name'] == name][0]
    if copy:
        monster = monster.copy()
    return monster

In [8]:
aboleth = getMonsterWithName('Aboleth')

In [9]:
draco = getMonsterWithName('Androsphinx')
draco

{'name': 'Androsphinx',
 'size': 'L',
 'type': 'monstrosity',
 'alignment': 'lawful neutral',
 'ac': '17 (natural armor)',
 'hp': '199 (19d10+95)',
 'speed': '40 ft., fly 60 ft.',
 'str': '22',
 'dex': '10',
 'con': '20',
 'int': '16',
 'wis': '18',
 'cha': '23',
 'save': 'Dex +6, Con +11, Int +9, Wis +10',
 'skill': 'Arcana +9, Perception +10, Religion +15',
 'resist': None,
 'vulnerable': None,
 'immune': 'psychic; bludgeoning, piercing, and slashing from nonmagical attacks',
 'conditionImmune': 'charmed, frightened',
 'senses': 'truesight 120 ft.',
 'passive': '20',
 'languages': 'Common, Sphinx',
 'cr': '17',
 'trait': [{'name': 'Source', 'text': 'Monster Manual p. 281'},
  {'name': 'Inscrutable',
   'text': "The sphinx is immune to any effect that would sense its emotions or read its thoughts, as well as any divination spell that it refuses. Wisdom (Insight) checks made to ascertain the sphinx's intentions or sincerity have disadvantage."},
  {'name': 'Magic Weapons',
   'text': "

# Initial Aboleth

In [10]:
aboleth

{'name': 'Aboleth',
 'size': 'L',
 'type': 'aberration',
 'alignment': 'lawful evil',
 'ac': '17 (natural armor)',
 'hp': '135 (18d10+36)',
 'speed': '10 ft., swim 40 ft.',
 'str': '21',
 'dex': '9',
 'con': '15',
 'int': '18',
 'wis': '15',
 'cha': '18',
 'save': 'Con +6, Int +8, Wis +6',
 'skill': 'History +12, Perception +10',
 'resist': None,
 'vulnerable': None,
 'immune': None,
 'conditionImmune': None,
 'senses': 'darkvision 120 ft.',
 'passive': '20',
 'languages': 'Deep Speech, telepathy 120 ft.',
 'cr': '10',
 'trait': [{'name': 'Source', 'text': 'Monster Manual p. 13'},
  {'name': 'Amphibious', 'text': 'The aboleth can breathe air and water'},
  {'name': 'Mucous Cloud',
   'text': 'While underwater, the aboleth is surrounded by transformative mucus. A creature that touches the aboleth or that hits it with a melee attack while within 5 ft. of it must make a DC 14 Constitution saving throw. On a failure, the creature is diseased for 1d4 hours. The diseased creature can breathe

# Convert Aboleth

In [11]:
aboleth['size'] = Size[aboleth['size']].value

In [12]:
hp_and_hit_dice = aboleth['hp'].split(' ')
aboleth['hp'] = hp_and_hit_dice[0]
aboleth['hit_dice'] = " ".join(hp_and_hit_dice[1:])

In [13]:
aboleth["stats"] = [
    aboleth["str"],
    aboleth["dex"],
    aboleth["con"],
    aboleth["int"],
    aboleth["wis"],
    aboleth["cha"],
]
_ = (
    aboleth.pop("str"),
    aboleth.pop("dex"),
    aboleth.pop("con"),
    aboleth.pop("int"),
    aboleth.pop("wis"),
    aboleth.pop("cha"),
)

In [14]:
saves = aboleth['save'].split(",")
new_saves = []
for save in saves:
    save = save.lstrip()
    stat, value = save.split(" ")
    value = int(value.replace('+', ""))
    stat = Stat[stat].value
    print(stat, value)
    new_saves.append({ stat: value})
new_saves
aboleth['saves'] = new_saves
aboleth.pop('save')

consitution 6
intelligence 8
wisdom 6


'Con +6, Int +8, Wis +6'

In [15]:
skillsaves = aboleth['skill'].split(",")
skillsaves
new_skillsaves = []
for save in skillsaves:
    save = save.lstrip()
    stat, value = save.split(" ")
    value = int(value.replace('+', ""))
    stat = stat.lower()
    print(stat, value)
    new_skillsaves.append({ stat: value})
new_skillsaves
aboleth['skillsaves'] = new_skillsaves
aboleth.pop('skill')

history 12
perception 10


'History +12, Perception +10'

In [16]:
aboleth['damage_resistances'] = aboleth['resist'] or ""
aboleth.pop('resist')

In [17]:
aboleth['damage_vulnerabilities'] = aboleth['vulnerable'] or ""
aboleth.pop('vulnerable')

In [18]:
aboleth['damage_immunities'] = aboleth['immune'] or ""
aboleth.pop('immune')

In [19]:
aboleth['condition_immunities'] = aboleth['conditionImmune'] or ""
aboleth.pop('conditionImmune')

In [20]:
aboleth['senses'] += ", passive Perception " + aboleth['passive']
aboleth.pop('passive')

'20'

In [21]:
aboleth['bestiary'] = True

In [22]:
sourceTrait = None
for trait in aboleth['trait']:
    if 'Source' in trait['name']:
        sourceTrait = trait
    elif trait.get('text') is not None:
        trait['desc'] = trait.pop('text')

aboleth['trait'].pop(aboleth['trait'].index(sourceTrait))

if sourceTrait:
    book = sourceTrait['text'].split(" p.")[0]
    aboleth['source'] = book
else:
    aboleth['source'] = "Not Found"

aboleth['traits'] = aboleth.pop('trait')

In [23]:
aboleth['actions'] = aboleth.pop('action')

for action in aboleth['actions']:
    text = action.get('text')
    if text and action.get('desc') is None:
        if type(text) == list:
            text = '\n'.join(text)
            action['text'] = text
        action['desc'] = action.pop('text')
    if action.get('attack') is not None:
        attack = action.pop('attack')
        _, attack_bonus, damage = attack.split('|')
        damages = damage.split('+')
        damage_dice_list = [damage_instance for damage_instance in damages if 'd' in damage_instance]
        damage_bonus_list = [damage_instance for damage_instance in damages if 'd' not in damage_instance]
        damage_dice = "".join(damage_dice_list)
        damage_bonus = damage_bonus_list[0]
        action['attack_bonus'] = attack_bonus
        action['damage_dice'] = damage_dice
        action['damage_bonus'] = damage_bonus

In [24]:
aboleth['legendary_actions'] = aboleth.pop('legendary')

for legendary in aboleth['legendary_actions']:
    text = legendary.get('text')
    if text and legendary.get('desc') is None:
        if type(text) == list:
            text = '\n'.join(text)
            legendary['text'] = text
        legendary['desc'] = legendary.pop('text')

In [25]:
aboleth.pop('spells')
aboleth.pop('slots')
aboleth.pop('environment')

'underdark'

In [26]:
aboleth

{'name': 'Aboleth',
 'size': 'Large',
 'type': 'aberration',
 'alignment': 'lawful evil',
 'ac': '17 (natural armor)',
 'hp': '135',
 'speed': '10 ft., swim 40 ft.',
 'senses': 'darkvision 120 ft., passive Perception 20',
 'languages': 'Deep Speech, telepathy 120 ft.',
 'cr': '10',
 'hit_dice': '(18d10+36)',
 'stats': ['21', '9', '15', '18', '15', '18'],
 'saves': [{'consitution': 6}, {'intelligence': 8}, {'wisdom': 6}],
 'skillsaves': [{'history': 12}, {'perception': 10}],
 'damage_resistances': '',
 'damage_vulnerabilities': '',
 'damage_immunities': '',
 'condition_immunities': '',
 'bestiary': True,
 'source': 'Monster Manual',
 'traits': [{'name': 'Amphibious',
   'desc': 'The aboleth can breathe air and water'},
  {'name': 'Mucous Cloud',
   'desc': 'While underwater, the aboleth is surrounded by transformative mucus. A creature that touches the aboleth or that hits it with a melee attack while within 5 ft. of it must make a DC 14 Constitution saving throw. On a failure, the crea

In [27]:
json.dump(aboleth, fp=open('aboleth_transformed.json', 'w'))